In [1]:
# 1. Imports
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

# 2. Load dataset
data_path = '/Users/admin/Library/CloudStorage/OneDrive-GettysburgCollege/DS325/Baza customer Telecom v2.csv' 
df = pd.read_csv(data_path)

# 3. Quick inspection
print("Dataset shape:", df.shape)
display(df.head())
df.info()
df.describe() 


Dataset shape: (8453, 14)


,PID,CRM_PID_Value_Segment,EffectiveSegment,Billing_ZIP,KA_name,Active_subscribers,Not_Active_subscribers,Suspended_subscribers,Total_SUBs,AvgMobileRevenue,AvgFIXRevenue,TotalRevenue,ARPU,CHURN
0,123759242,Bronze,SOHO,6000.0,VM,2,NaN,NaN,2,40.17,0.0,40.17,NaN,No
1,126145737,Bronze,SOHO,6400.0,VM,3,NaN,NaN,3,40.17,0.0,40.17,13.39,No
2,123506355,Bronze,SOHO,6000.0,DI,2,3.0,NaN,5,40.17,0.0,40.17,20.09,No
3,112595585,Bronze,SOHO,4400.0,MT,1,2.0,NaN,3,40.17,0.0,40.17,40.17,No
4,115097935,Iron,SOHO,4000.0,AD,2,1.0,NaN,3,40.17,0.0,40.17,20.09,No


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8453 entries, 0 to 8452
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   PID                     8453 non-null   object 
 1   CRM_PID_Value_Segment   8448 non-null   object 
 2   EffectiveSegment        8453 non-null   object 
 3   Billing_ZIP             8451 non-null   float64
 4   KA_name                 8453 non-null   object 
 5   Active_subscribers      8453 non-null   int64  
 6   Not_Active_subscribers  4304 non-null   float64
 7   Suspended_subscribers   352 non-null    float64
 8   Total_SUBs              8453 non-null   int64  
 9   AvgMobileRevenue        8453 non-null   float64
 10  AvgFIXRevenue           8453 non-null   float64
 11  TotalRevenue            8453 non-null   float64
 12  ARPU                    8452 non-null   float64
 13  CHURN                   8453 non-null   object 
dtypes: float64(7), int64(2), object(5)
memor

,Billing_ZIP,Active_subscribers,Not_Active_subscribers,Suspended_subscribers,Total_SUBs,AvgMobileRevenue,AvgFIXRevenue,TotalRevenue,ARPU
count,8451.000000,8453.000000,4304.000000,352.000000,8453.000000,8453.000000,8453.000000,8453.000000,8452.000000
mean,4879.727725,7.774636,4.163336,1.576705,9.960132,148.011956,0.821185,148.833141,24.441789
std,1061.095394,6.680524,9.462847,1.979905,10.246648,102.570539,11.737880,103.250779,22.820585
min,1000.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,4.670000,0.000000
25%,4003.000000,4.000000,1.000000,1.000000,5.000000,71.500000,0.000000,71.830000,14.070000
50%,4400.000000,6.000000,2.000000,1.000000,7.000000,113.170000,0.000000,113.670000,19.315000
75%,6000.000000,10.000000,4.000000,1.000000,12.000000,191.170000,0.000000,192.330000,27.255000
max,9644.000000,110.000000,214.000000,22.000000,235.000000,499.830000,480.500000,499.830000,462.830000


In [2]:
# 4. Missing Data Inspection & Imputation

# Inspect missing values
missing = df.isnull().sum()
print("Missing values before imputation:")
display(missing)

# Impute subscriber count nulls with 0
df['Not_Active_subscribers'] = df['Not_Active_subscribers'].fillna(0)
df['Suspended_subscribers'] = df['Suspended_subscribers'].fillna(0)

# Handle ARPU nulls by recalculating where possible
computed_arpu = df['TotalRevenue'] / df['Active_subscribers'].replace(0, pd.NA)
df['ARPU'] = df['ARPU'].fillna(computed_arpu).fillna(0)

# Convert CHURN to numeric 0/1
df['CHURN'] = df['CHURN'].map({'No': 0, 'Yes': 1})

# Verify changes
print("\nMissing values after imputation:")
display(df.isnull().sum())

print("\nData types and sample values:")
display(df[['Not_Active_subscribers','Suspended_subscribers','ARPU','CHURN']].head())



Missing values before imputation:


PID                          0
CRM_PID_Value_Segment        5
EffectiveSegment             0
Billing_ZIP                  2
KA_name                      0
Active_subscribers           0
Not_Active_subscribers    4149
Suspended_subscribers     8101
Total_SUBs                   0
AvgMobileRevenue             0
AvgFIXRevenue                0
TotalRevenue                 0
ARPU                         1
CHURN                        0
dtype: int64


Missing values after imputation:


PID                       0
CRM_PID_Value_Segment     5
EffectiveSegment          0
Billing_ZIP               2
KA_name                   0
Active_subscribers        0
Not_Active_subscribers    0
Suspended_subscribers     0
Total_SUBs                0
AvgMobileRevenue          0
AvgFIXRevenue             0
TotalRevenue              0
ARPU                      0
CHURN                     0
dtype: int64


Data types and sample values:


,Not_Active_subscribers,Suspended_subscribers,ARPU,CHURN
0,0.0,0.0,20.085,0
1,0.0,0.0,13.390,0
2,3.0,0.0,20.090,0
3,2.0,0.0,40.170,0
4,1.0,0.0,20.090,0


In [3]:
# 4.b Clean column names (strip whitespace)
df.columns = df.columns.str.strip()

# 5. Sanity Checks: Subscriber and Revenue Consistency

# 5.1 Subscriber totals check
df['subs_sum'] = (
    df['Active_subscribers'] +
    df['Not_Active_subscribers'] +
    df['Suspended_subscribers']
)
subs_mismatch = df['subs_sum'] != df['Total_SUBs']
subs_mismatch_count = subs_mismatch.sum()

# 5.2 Revenue components check
# Round to cents for comparison
mobile_fix_sum = (df['AvgMobileRevenue'] + df['AvgFIXRevenue']).round(2)
total_rev = df['TotalRevenue'].round(2)
rev_mismatch = mobile_fix_sum != total_rev
rev_mismatch_count = rev_mismatch.sum()

# Display summary
print(f"Subscriber count mismatches: {subs_mismatch_count} of {len(df)}")
print(f"Revenue sum mismatches: {rev_mismatch_count} of {len(df)}")

# Show a few examples of each mismatch type
if subs_mismatch_count > 0:
    display(df.loc[subs_mismatch, 
                   ['PID', 'Active_subscribers', 'Not_Active_subscribers', 
                    'Suspended_subscribers', 'Total_SUBs', 'subs_sum']].head())

if rev_mismatch_count > 0:
    display(df.loc[rev_mismatch, 
                   ['PID', 'AvgMobileRevenue', 'AvgFIXRevenue', 
                    'TotalRevenue']].head())



Subscriber count mismatches: 0 of 8453
Revenue sum mismatches: 0 of 8453


In [4]:
# 7. Finalizing Type Conversions & Preprocessor Definition

# Handle missing Billing_ZIP and create ZIP_region
df['Billing_ZIP'] = df['Billing_ZIP'].fillna(0).astype(int).astype(str)
df['ZIP_region'] = df['Billing_ZIP'].str[:2]

# Define feature lists
categorical_features = [
    'CRM_PID_Value_Segment', 
    'EffectiveSegment', 
    'KA_name', 
    'ZIP_region'
]
numeric_features = [
    'Active_subscribers', 
    'Not_Active_subscribers', 
    'Suspended_subscribers', 
    'Total_SUBs',
    'AvgMobileRevenue', 
    'AvgFIXRevenue', 
    'TotalRevenue', 
    'ARPU'
]

# Convert to categorical dtype
df[categorical_features] = df[categorical_features].astype('category')

# Build preprocessing pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

numeric_pipeline = Pipeline([
    ('scaler', StandardScaler())
])
from sklearn.preprocessing import OneHotEncoder

categorical_pipeline = Pipeline([
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', numeric_pipeline, numeric_features),
    ('cat', categorical_pipeline, categorical_features)
])


# Preview: print the transformer names and feature counts
print("Transformer steps:")
print(preprocessor)



Transformer steps:
ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('scaler', StandardScaler())]),
                                 ['Active_subscribers',
                                  'Not_Active_subscribers',
                                  'Suspended_subscribers', 'Total_SUBs',
                                  'AvgMobileRevenue', 'AvgFIXRevenue',
                                  'TotalRevenue', 'ARPU']),
                                ('cat',
                                 Pipeline(steps=[('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['CRM_PID_Value_Segment', 'EffectiveSegment',
                                  'KA_name', 'ZIP_region'])])


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.pipeline import Pipeline

# 8. Prepare features and target
feature_cols = numeric_features + categorical_features
X = df[feature_cols]
y = df['CHURN']

RANDOM_STATE = 42 

# 9. Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=RANDOM_STATE
)

# 10. Logistic Regression Pipeline
clf_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(solver='liblinear', random_state=RANDOM_STATE))
])

# 11. Fit the model
clf_pipeline.fit(X_train, y_train)

# 12. Predictions and Probabilities
y_pred = clf_pipeline.predict(X_test)
y_proba = clf_pipeline.predict_proba(X_test)[:, 1]

# 13. Evaluation Metrics
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=['No Churn', 'Churn']))

roc_auc = roc_auc_score(y_test, y_proba)
print(f"ROC AUC Score: {roc_auc:.3f}")

# 14. Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(cm)


Classification Report:
              precision    recall  f1-score   support

    No Churn       0.93      1.00      0.97      1581
       Churn       0.00      0.00      0.00       110

    accuracy                           0.93      1691
   macro avg       0.47      0.50      0.48      1691
weighted avg       0.87      0.93      0.90      1691

ROC AUC Score: 0.580

Confusion Matrix:
[[1581    0]
 [ 110    0]]


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

# 1. Logistic Regression with class_weight='balanced'
clf_balanced = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(
        solver='liblinear', 
        random_state=RANDOM_STATE, 
        class_weight='balanced'
    ))
])

# 2. Fit the balanced model
clf_balanced.fit(X_train, y_train)

# 3. Predictions and Probabilities
y_pred_bal = clf_balanced.predict(X_test)
y_proba_bal = clf_balanced.predict_proba(X_test)[:, 1]

# 4. Evaluation Metrics
print("Classification Report (Balanced Logistic Regression):")
print(classification_report(y_test, y_pred_bal, target_names=['No Churn', 'Churn'], zero_division=0))

roc_auc_bal = roc_auc_score(y_test, y_proba_bal)
print(f"ROC AUC Score: {roc_auc_bal:.3f}")

# 5. Confusion Matrix
cm_bal = confusion_matrix(y_test, y_pred_bal)
print("\nConfusion Matrix:")
print(cm_bal)


Classification Report (Balanced Logistic Regression):
              precision    recall  f1-score   support

    No Churn       0.95      0.60      0.73      1581
       Churn       0.08      0.53      0.14       110

    accuracy                           0.59      1691
   macro avg       0.52      0.56      0.44      1691
weighted avg       0.89      0.59      0.70      1691

ROC AUC Score: 0.572

Confusion Matrix:
[[947 634]
 [ 52  58]]
